# Feature Extraction using HOG, PCA and ResNet
* Load Manifest and Functions
* Load Images Function
* EXTRACTOR 1 — HOG (Histogram of Oriented Gradients) FEATURES
* EXTRACTOR 2 — PCA (Principle Component analysis) Features
* EXTRACTOR 3 — ResNet50 Deep Features

In [ ]:
import os   ### For working of CUDA 

# Disable XLA completely
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/local/cuda"
os.environ["TF_DISABLE_XLA"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import pandas as pd
import numpy as np
import cv2
import os
from pathlib import Path
from tqdm import tqdm
from skimage.feature import hog
from sklearn.decomposition import PCA
import tensorflow as tf
tf.config.optimizer.set_jit(False)
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

In [3]:
BASE_DIR = Path().resolve().parents[0]
csv_path = BASE_DIR / 'data' / 'processed' / 'manifest.csv'

df = pd.read_csv(csv_path)

df.head()

,image_path,label,video_id,split
0,/home/aryan/code/DeepFakeDetection/data/proces...,0,183,train
1,/home/aryan/code/DeepFakeDetection/data/proces...,0,183,train
2,/home/aryan/code/DeepFakeDetection/data/proces...,0,183,train
3,/home/aryan/code/DeepFakeDetection/data/proces...,0,183,train
4,/home/aryan/code/DeepFakeDetection/data/proces...,0,183,train


## Load Images Function


In [4]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    return img


## EXTRACTOR 1 — HOG (Histogram of Oriented Gradients) FEATURES
* Give featues --> edges, gradients and shapes

In [5]:
def extract_hog_features(df):
    hog_features = []
    
    for path in tqdm(df['image_path'], desc="Extracting HOG"):
        img = load_image(path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  ## Hog works best with Greyscale images
        
        features = hog(gray,
                       orientations=9,              ## Standard Practise
                       pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2),
                       block_norm='L2-Hys')
        
        hog_features.append(features)
        
    return np.array(hog_features)  ## Convert to numpy for model input


In [6]:
hog_X = extract_hog_features(df)
hog_y = df['label'].values

hog_X.shape  ## Rows and columns


Extracting HOG: 100%|██████████| 1413/1413 [00:15<00:00, 93.74it/s]


(1413, 26244)

In [12]:
np.save("../data/features/hog_features.npy", hog_X)
np.save("../data/features/hog_labels.npy", hog_y)


## EXTRACTOR 2 — PCA (Principle Component analysis) Features
* Helps show that classical ML struggles with raw pixel PCA

In [7]:
def extract_pca_features(df, n_components=150):  ## How many principle componets to keep
    X = []
    
    for path in tqdm(df['image_path'], desc="Loading images for PCA"):
        img = load_image(path)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) ## PCA to greyscale removes color channel further reducing the dimensionality
        img = img.flatten() ## Converts 2D vector to 1D vector 
        X.append(img)
    
    X = np.array(X)
    
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X)
    
    return X_pca, pca


In [8]:
pca_X, pca_model = extract_pca_features(df)
pca_y = df['label'].values
pca_X.shape


Loading images for PCA: 100%|██████████| 1413/1413 [00:00<00:00, 3851.23it/s]


(1413, 150)

In [13]:
np.save("../data/features/pca_features.npy", pca_X)
np.save("../data/features/pca_labels.npy", pca_y)


## EXTRACTOR 3 — ResNet50 Deep Features
* Extracts high-level semantic features
* Best classical model results
* Shows why deep learning is superior

In [9]:
resnet = ResNet50(weights="imagenet", include_top=False, pooling='avg')

def extract_resnet_features(df):
    deep_features = []

    for path in tqdm(df['image_path'], desc="Extracting ResNet Features"):
        img = load_image(path)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        feat = resnet.predict(img, verbose=0)
        deep_features.append(feat.flatten())

    return np.array(deep_features)



I0000 00:00:1764463303.025225   12146 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2605 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [10]:
resnet_X = extract_resnet_features(df)
resnet_y = df['label'].values

resnet_X.shape


Extracting ResNet Features: 100%|██████████| 1413/1413 [01:39<00:00, 14.16it/s]


(1413, 2048)

In [14]:
np.save("../data/features/resnet_features.npy", resnet_X)
np.save("../data/features/resnet_labels.npy", resnet_y)
